# Lesson 4: detecting general disturbances using HLS datasets

**Author: Yingchu Hu, Su Ye (remotesensingsuy@gmail.com)**

**Time series datasets: Harmonized Landsat-Sentinel (HLS) datasets**

**Application: general disturbance detection**

This tutorial will guide you through processing tile-based time-series images using pyxccd to generate annual disturbance maps, recent disturbance maps, and first disturbance maps. The tile-based processing workflow typically consists of three main steps:

Step 1: **image stacking**. 

This step physically converts the original HLS images into several hundred time-series blocks to maximize data locality. Block dimensions are specified in `config_hls.yaml`. By storing data as smaller, spatially contiguous blocks, each compute thread or node operates on a local subset of data, avoiding contention on a single shared source and enabling parallel read/write operations. This reduces disk and network I/O bottlenecks during subsequent time-series processing and significantly speeds up ingestion. The stacked output is saved in a Python-native `.npy` format. During stacking, bit-encoded QA flags are converted to integer class codes (0 = clear, 1 = water, 2 = shadow, 3 = snow, 4 = cloud);

Step 2: **time-series processing**. 

Process the stacked blocks with a time-series algorithm (COLD or S-CCD). Algorithm parameters are configurable in `config_hls`.yaml. For each block, the step outputs block-based per-segment information (for every detected segment): break time, start time, harmonic coefficients, RMSE, and other diagnostic fields. The goal is to record comprehensive segment metadata so that downstream map generation (disturbance maps, coefficient maps, classification maps) can be produced without re-processing raw images. Processing is parallelizable across threads or nodes; users should adapt the provided desktop parallelization example to their local environment (cluster, SLURM, Dask, Spark, etc.).


Step 3: **map generation**. 

Generate maps from the per-segment outputs according to user needs. Example outputs include annual disturbance maps and ensemble disturbance maps that summarize location, disturbance category, and timing. Because segment metadata are comprehensive, map generation is I/O-light and can be performed rapidly once segment files are available.
---

## Step 0: preparation

### 1. Install Pyxccd
```
pip install pyxccd
```

### 2. Prepare HLS images
Prepare downloaded HLS files with the following structure (example contains 6 years of HLS data from 2019-2024):  
Quark Drive download link: https://pan.quark.cn/s/f6456bf1ad27?pwd=FxtU  
Extraction code: FxtU
```
HLS_root_directory/
└── Tile_ID (e.g., 51RTP)/
    └── HLS_image_files (e.g., HLS.S30.T51RTP.2021001.v2.0.B02.tif)
```

### 3. Configuration File
Prepare `config_hls.yaml` with adjustable block size parameters (example uses 30×30 blocks):
```yaml
DATASETINFO:
  n_rows: 3660
  n_cols: 3660
  n_block_x: 30
  n_block_y: 30
```

## Step 1: Image Stacking

### Procedure
1. Ensure tile list file `tile_list` contains target tile IDs (one per line, example uses 51RTP)

2. Run stacking script:
```bash
python step1_stack.py --tile_list_fn ./tile_list \
                     --meta_path /path/to/hls/data \
                     --yaml_path ./config_hls.yaml \
                     --out_path /stack \
                     --low_date_bound 2019-01-01 \
                     --upp_date_bound 2024-12-31 \
                     --n_cores 16
```

### Parameters
```
--tile_list_fn: Path to tile list file
--meta_path: HLS data root directory
--yaml_path: Configuration file path
--out_path: Output directory for stacked data
--low_date_bound: Start date (YYYY-MM-DD)
--upp_date_bound: End date (YYYY-MM-DD)
--n_cores: Number of CPU cores to use
```

### Output
The output directory (`stack` by default) will contain `{TileID}_stack` folders with block-organized stacked data for each tile.

## Step 2: Time-series processing (SCCD & COLD)

#### Procedure
1. Complete the stacking process (Step 1)
2. Run SCCD detection script:
```bash
python step2_sccd.py --tile_list_fn ./tile_list \
                     --stack_path /stack \
                     --result_parent_path /sccd_results \
                     --yaml_path /config_hls.yaml \
                     --low_datebound 2019-01-01 \
                     --upper_datebound 2024-12-31 \
                     --n_cores 16
```

#### Parameters
```
--stack_path: Stacked data directory from Step 1
--result_parent_path: Output directory for SCCD results
(Other parameters same as Step 1)
```

#### Output
The output directory (`sccd_results` by default) will contain:
```
record_change_x{blockX}_y{blockY}_sccd.npy: Change detection results per block
SCCD_block{blockID}_finished.txt: Completion marker files
```

### COLD Algorithm

#### Procedure
1. Complete the stacking process (Step 1)
2. Run COLD detection script:
```bash
python step2_cold.py --tile_list_fn ./tile_list \
                     --stack_path /stack \
                     --result_parent_path /cold_results \
                     --yaml_path ./config_hls.yaml \
                     --low_datebound 2019-01-01 \
                     --upper_datebound 2024-12-31 \
                     --n_cores 16
```

#### Parameters
```
--stack_path: Stacked data directory from Step 1
--result_parent_path: Output directory for COLD results
(Other parameters same as Step 1)
```

#### Output
The output directory (`cold_results` by default) will contain:
```
record_change_x{blockX}_y{blockY}_cold.npy: Change detection results per block
COLD_block{blockID}_finished.txt: Completion marker files
```

## Step 3: Map generation

### Purpose
Convert change detection results into annual/recent/first disturbance maps.

### Procedure
1. Complete Steps 1-2
2. Run disturbance mapping script:
```bash
python step3_disturbance_map.py --source_dir /hls \
                               --result_path /sccd_results/51RTP_sccd \
                               --out_path /disturbance_maps \
                               --yaml_path /config_hls.yaml \
                               --year_lowbound 2019 \
                               --year_uppbound 2024 \
                               --n_cores 16
```

### Parameters
```
--source_dir: HLS root directory (for spatial reference)
--result_path: SCCD results directory (specific to tile)
--out_path: Output directory for disturbance maps
--year_lowbound: Start year
--year_uppbound: End year
--n_cores: Number of CPU cores
```

### Output
The output directory (`disturbance_maps` by default) will contain:
```
{year}_break_map_SCCDOFFLINE.tif: Annual disturbance map
recent_disturbance_map_SCCDOFFLINE.tif: Recent disturbance map (year of latest disturbance)
first_disturbance_map_SCCDOFFLINE.tif: First disturbance map (year of first disturbance)
```

### Interpretation
#### Annual Disturbance Maps
Pixel value = disturbance_type × 1000 + day_of_year  
Disturbance types:  
&nbsp;&nbsp;1 - Vegetation disturbance  
&nbsp;&nbsp;2 - Non-vegetation disturbance


#### Recent Disturbance Map
Shows the most recent disturbance year for each pixel  
0 indicates no disturbance

#### First Disturbance Map
Shows the first disturbance year for each pixel  
0 indicates no disturbance


## Important Notes
1. For large areas, process tiles in batches to avoid memory overload
2. Adjust block size parameters in config_hls.yaml to balance speed and memory usage
3. Interrupted processing can be resumed - the script will skip completed blocks


## Example Output
2019-2024 First Disturbance Map (SCCD)

![First Disturbance Map](first_disturb1.png)